In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Joins and Data Partitions").getOrCreate()
)

In [2]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

# emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"
emp_schema = ["emp_id", "dept_id", "name", "age", "gender", "salary", "hire_date"]

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

# dept_schema = "department_id string, department_name string, city string, country string, budget string"
dept_schema = ["dept_id", "dept_name", "city", "country", "budget"]

In [3]:
# Create emp & dept DataFrame
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [4]:
emp.show()
dept.show()

+------+-------+-------------+---+------+------+----------+
|emp_id|dept_id|         name|age|gender|salary| hire_date|
+------+-------+-------------+---+------+------+----------+
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|   006|    103|    Jill Wong| 32|Female| 52000|2018-07-01|
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|
|   009|    103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|   011|    104|   David Park| 38|  Male| 65000|2015-11-01|
|   012|    105|   Susan Chen| 31|Female| 54000|2017-02-15|
|   013|    106|    Brian Kim| 45|  Male| 75000|2011-07-01|
|   014|    107|    Emily Lee| 26|Female

In [5]:
# Print Schema
emp.printSchema()
dept.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

root
 |-- dept_id: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- budget: string (nullable = true)



In [6]:
# get number of partitions for emp dataframe
emp.rdd.getNumPartitions()

4

In [7]:
# get number of partitions for dept dataframe
dept.rdd.getNumPartitions()

4

In [19]:
# Repartition of data using repartition & coalesce
emp_partitioned = emp.repartition(8)  # default was 4 as the number of vCPU or cores of your host machine.
dept_partitioned = dept.coalesce(3)  # coalesce with only decrease the number of partition from defautl value.

In [20]:
emp_partitioned.rdd.getNumPartitions()

8

In [21]:
dept_partitioned.rdd.getNumPartitions()

3

In [22]:
# Repartition of data using repartition and based on column name
emp_partitioned = emp.repartition(6, "dept_id")

In [23]:
emp_partitioned.rdd.getNumPartitions()

6

In [27]:
# Find the partition info for partitions and reparition
from pyspark.sql.functions import spark_partition_id

emp_1 = emp.withColumn("partition_num", spark_partition_id())

In [28]:
emp_1.show()

+------+-------+-------------+---+------+------+----------+-------------+
|emp_id|dept_id|         name|age|gender|salary| hire_date|partition_num|
+------+-------+-------------+---+------+------+----------+-------------+
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|            0|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|            0|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|            0|
|   006|    103|    Jill Wong| 32|Female| 52000|2018-07-01|            1|
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|            1|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|            1|
|   009|    103|      Tom Tan| 33|  Male| 58000|2016-06-01|            1|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|            1|
|   011|    104|   David Park| 38|  Ma

In [31]:
# Find the partition info for partitions and reparition
from pyspark.sql.functions import spark_partition_id

emp_1 = emp.repartition(4, "dept_id").withColumn("partition_num", spark_partition_id())  # partition based on dept_id column. this will bring all the dept ids under same partition

In [32]:
emp_1.show()

+------+-------+-------------+---+------+------+----------+-------------+
|emp_id|dept_id|         name|age|gender|salary| hire_date|partition_num|
+------+-------+-------------+---+------+------+----------+-------------+
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|   014|    107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|   016|    107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|   020|    102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|   012|    105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|   017|    105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|            2|
|   011|    104|   David Park| 38|  Male| 65000|2015-11-01|            2|
|   013|    106|    Brian Kim| 45|  Ma

In [33]:
# INNER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e inner join dept d on emp.department_id = dept.department_id

df_joined = emp.alias("e").join(dept.alias("d"), how="inner", on=emp.dept_id==dept.dept_id)

In [34]:
df_joined.select("e.name", "d.dept_name", "d.dept_id", "e.salary").show()

+-------------+--------------------+-------+------+
|         name|           dept_name|dept_id|salary|
+-------------+--------------------+-------+------+
|     John Doe|               Sales|    101| 50000|
|   Jane Smith|               Sales|    101| 45000|
|James Johnson|               Sales|    101| 70000|
|    Bob Brown|           Marketing|    102| 55000|
|    Alice Lee|           Marketing|    102| 48000|
|     Kate Kim|           Marketing|    102| 51000|
|    Grace Kim|           Marketing|    102| 53000|
|    Jack Chan|             Finance|    103| 60000|
|    Jill Wong|             Finance|    103| 52000|
|      Tom Tan|             Finance|    103| 58000|
|  Steven Chen|             Finance|    103| 62000|
|     Lisa Lee|         Engineering|    104| 47000|
|   David Park|         Engineering|    104| 65000|
|    Nancy Liu|         Engineering|    104| 50000|
|   Susan Chen|     Human Resources|    105| 54000|
|  George Wang|     Human Resources|    105| 57000|
|    Brian K

In [35]:
# LEFT OUTER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e left outer join dept d on emp.department_id = dept.department_id

df_joined = emp.alias("e").join(dept.alias("d"), how="left_outer", on=emp.dept_id==dept.dept_id)

In [37]:
df_joined.select("e.name", "d.dept_name", "d.dept_id", "e.salary").show()

+-------------+--------------------+-------+------+
|         name|           dept_name|dept_id|salary|
+-------------+--------------------+-------+------+
|     John Doe|               Sales|    101| 50000|
|   Jane Smith|               Sales|    101| 45000|
|    Bob Brown|           Marketing|    102| 55000|
|    Alice Lee|           Marketing|    102| 48000|
|    Jack Chan|             Finance|    103| 60000|
|James Johnson|               Sales|    101| 70000|
|     Lisa Lee|         Engineering|    104| 47000|
|     Kate Kim|           Marketing|    102| 51000|
|    Jill Wong|             Finance|    103| 52000|
|      Tom Tan|             Finance|    103| 58000|
|    Emily Lee|    Customer Service|    107| 46000|
|   David Park|         Engineering|    104| 65000|
|    Brian Kim|Research and Deve...|    106| 75000|
|  Michael Lee|Research and Deve...|    106| 63000|
|   Susan Chen|     Human Resources|    105| 54000|
|  Kelly Zhang|    Customer Service|    107| 49000|
|    Nancy L

In [46]:
# Bonus TIP
# Joins with cascading conditions
# Join with Department_id and only for departments 101 or 102
# Join with not null/null conditions

df_final = emp.join(dept, how="left_outer", 
                   on=(emp.dept_id == dept.dept_id) & ((emp.dept_id == "101") | (emp.dept_id == "102")) 
                    & (emp.dept_id.isNotNull())
                   )

In [47]:
df_final.show()

+------+-------+-------------+---+------+------+----------+-------+---------+----+-------+-------+
|emp_id|dept_id|         name|age|gender|salary| hire_date|dept_id|dept_name|city|country| budget|
+------+-------+-------------+---+------+------+----------+-------+---------+----+-------+-------+
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|    101|    Sales| NYC|     US|1000000|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|    101|    Sales| NYC|     US|1000000|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|    102|Marketing|  LA|     US| 900000|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|    102|Marketing|  LA|     US| 900000|
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|   NULL|     NULL|NULL|   NULL|   NULL|
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|    101|    Sales| NYC|     US|1000000|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|   NULL|     NULL|NULL|   NULL|   NULL|
|   008|  

In [48]:
spark.stop()